In [1]:
from json import JSONDecoder

In [2]:
def read_JSON_as_list(filename):
    def parse_pairs(pairs):
        return pairs

    decoder = JSONDecoder(object_pairs_hook=parse_pairs)
    with open(filename) as json_file:
        file_content = json_file.read()
        data = decoder.decode(file_content)
        
    return data

In [3]:
data = read_JSON_as_list('./JSON/mental_imagery_extended.json')

# Get all object instances

In [4]:
def get_all_objects(json_as_list):
    # first get the objects outside of groups
    objects = json_as_list[1][1]
    
    key_to_pos_dict = {}
    for ind, elem in enumerate(json_as_list):
        key_to_pos_dict[elem[0]] = ind
        
    groups = json_as_list[key_to_pos_dict['groups']][1]
    
    for group in groups:
        for obj in group[1][1][1]:
            # add the group name to the object, otherwise there might be duplicate object names
            # also added in connections
            obj[1][0] = ('name', group[0] + "." + obj[1][0][1])
            objects.append(obj)
            
    return objects

In [5]:
objects = get_all_objects(data)

In [6]:
''' data[1][1] has the value for 'steps' --> a list of the different objects 
    (except for the ones in groups)
    e.g. data[1][1][0] is a NeuralField, its first element is the type of object,
    i.e. in this case the string 'cedar.dynamics.NeuralField', its second element
    contains the parameters and their values
'''
print(objects[0][0], "\n")

print(objects[0][1], "\n")

cedar.dynamics.NeuralField 

[('name', 'Above Memory'), ('activation as output', 'false'), ('discrete metric (workaround)', 'false'), ('update stepIcon according to output', 'true'), ('threshold for updating the stepIcon', '0.80000000000000004'), ('dimensionality', '0'), ('sizes', ''), ('time scale', '100'), ('resting level', '-5'), ('input noise gain', '0.10000000000000001'), ('sigmoid', [('type', 'cedar.aux.math.AbsSigmoid'), ('threshold', '0'), ('beta', '100')]), ('global inhibition', '-0.01'), ('lateral kernels', [('cedar.aux.kernel.Box', [('dimensionality', '1'), ('anchor', ['0']), ('amplitude', '6.0999999999999996'), ('widths', ['2'])])]), ('lateral kernel convolution', [('borderType', 'Zero'), ('mode', 'Same'), ('engine', [('type', 'cedar.aux.conv.OpenCV')]), ('alternate even kernel center', 'false')]), ('noise correlation kernel', [('dimensionality', '1'), ('anchor', ['0']), ('amplitude', '0'), ('sigmas', ['3']), ('normalize', 'true'), ('shifts', ['0']), ('limit', '5')]), ('com

In [7]:
# check if I have all objects now by checking if the numbers are the same as
# in the json file
object_counts = {}

for obj in objects:
    if obj[0] not in object_counts.keys():
        object_counts[obj[0]] = 1
    else:
        object_counts[obj[0]] += 1
        
for key in object_counts:
    print('Number of %s: %i' %(key, object_counts[key]))

# --> Numbers check out

Number of cedar.dynamics.NeuralField: 90
Number of cedar.processing.sources.Boost: 23
Number of cedar.processing.ComponentMultiply: 52
Number of cedar.processing.sources.ConstMatrix: 8
Number of cedar.processing.steps.Convolution: 4
Number of cedar.processing.Flip: 4
Number of cedar.processing.sources.GaussInput: 16
Number of cedar.processing.Projection: 38
Number of cedar.processing.sources.SpatialTemplate: 24
Number of cedar.processing.StaticGain: 166


# Get all connections
For the groups the connections to and from the input nodes of the groups and output nodes of the groups have to be handled seperately, since these should be replaced by direct connections between the instances.

The names of the input and output nodes for groups are in the group element "connectors". If they have the value "true" they are input nodes, if the have the value "false" they are output nodes. In connections to input nodes of a group the target name is of the form: group_name.input_node_name


In [8]:
def add_group_name(group_connection, group_name):
    new_group_connection = [('source', group_name + "." + group_connection[0][1]),
                            ('target', group_name + "." + group_connection[1][1])]
    return new_group_connection


def find_connector_connections(connectors, 
                               outside_group_connections, 
                               inside_group_connections,
                               output_nodes=True):
    ''' outside_group_connections list of connections, inside_group_connections
        dictionary of list of connections per group.
    
        output_nodes True means that the connectors in the list are output
        nodes of their groups, if it is False they are input nodes.
        Elements in connectors should be of the form: 
        (group_name, connector_name)
    '''
    connector_connections = {}

    for connector in connectors:
        connector_connections[connector] = {'input connections': [], 
                          'output connections': []}

        if output_nodes:
            source_string = connector[0] + "." + connector[1]
            target_string = connector[0] + "." + connector[1] + ".input"
        else:
            source_string = connector[0] + "." + connector[1] + ".output"
            target_string = connector[0] + "." + connector[1] 

        # need the ":", otherwise removing and iterating at the same time leads to 
        # weird results
        for connection in outside_group_connections[:]:
            if connection[0][1] == source_string:
                connector_connections[connector]['output connections'].append(connection) 
                outside_group_connections.remove(connection)
            elif connection[1][1] == target_string:
                connector_connections[connector]['input connections'].append(connection)
                outside_group_connections.remove(connection)

        for connection in inside_group_connections[connector[0]][:]:
            if connection[1][1] == target_string:
                connector_connections[connector]['input connections'].append(connection)
                inside_group_connections[connector[0]].remove(connection)
            elif connection[0][1] == source_string:
                connector_connections[connector]['output connections'].append(connection)
                inside_group_connections[connector[0]].remove(connection)
                
    return connector_connections


def replace_middle_connection(middle_connection_dict):
    ''' takes as input a dictionary with input connections to a node and 
        output connections from the node and creates new connections substituting
        the middle node and connecting the inputs from the input nodes and the 
        outputs from the output nodes directly.
    '''
    # each input has to be connected to each output
    connections_list = []

    # go through all inputs
    for inp_connection in middle_connection_dict['input connections']:
        # source at position 0
        source = inp_connection[0]
        # go through all outputs
        for out_connection in middle_connection_dict['output connections']:
            # target at position 1
            target = out_connection[1]
            connections_list.append([source, target])

    return connections_list

In [9]:
def get_all_connections(json_as_list):
    
    key_to_pos_dict = {}
    for ind, elem in enumerate(data):
        key_to_pos_dict[elem[0]] = ind
        
    connections_outside_groups = json_as_list[key_to_pos_dict["connections"]][1]
    groups = json_as_list[key_to_pos_dict["groups"]][1]

    # get the connectors (input and output nodes of groups) and all group connections
    connectors = {}
    group_connections = {}

    for group in groups:
        # get the group name
        group_name = group[0]
        key_to_pos_dict_group = {}
        for ind, elem in enumerate(group[1]):
            key_to_pos_dict_group[elem[0]] = ind
        # get the connectors of this group
        connectors[group_name] = group[1][key_to_pos_dict_group["connectors"]][1]
        # get the connections of this group and add the group name to the 
        # object's name
        add_group_name_lambda = lambda x: add_group_name(x, group_name)
        group_connections_wn = list(map(add_group_name_lambda, group[1][key_to_pos_dict_group["connections"]][1]))
        group_connections[group_name] = group_connections_wn

    # divide connectors into input and output nodes
    group_outputs = []
    group_inputs = []

    for group in groups:
        for connector in connectors[group[0]]:
            if connector[1] == 'true':
                group_inputs.append((group[0], connector[0]))
            else:
                group_outputs.append((group[0], connector[0]))

    # get all connections to and from group output nodes
    group_output_connections = find_connector_connections(group_outputs, 
                                                          connections_outside_groups,
                                                          group_connections)
    # replace source -> group_output -> target connections by source -> target connections
    replaced_connections = list(map(replace_middle_connection, 
                                    group_output_connections.values()))
    # add the replaced connections to the connection list
    _ = [connections_outside_groups.extend(cons) for cons in replaced_connections]

    # now do the same for the group input nodes
    group_input_connections = find_connector_connections(group_inputs, 
                                                          connections_outside_groups,
                                                          group_connections,
                                                          output_nodes=False)
    replaced_connections = list(map(replace_middle_connection,
                                    group_input_connections.values()))
    _ = [connections_outside_groups.extend(cons) for cons in replaced_connections]

    # finally add the connections inside the groups now that all connections
    # to group input and output nodes have been removed
    _ = [connections_outside_groups.extend(cons) for cons in group_connections.values()]

    return connections_outside_groups

def number_of_connections(con_dict):
    length = 0
    if type(con_dict) == dict:
        for key in con_dict:
            length += len(con_dict[key])
    elif type(con_dict) == list:
        for sublist in con_dict:
            length += len(sublist)
    return length

In [10]:
all_connections = get_all_connections(data)

In [11]:
len(all_connections)

669

# Test simpler JSON file
To be sure that everything works correctly I added a small test architecture to the JSON files. It includes all scenarios of different group input and output connections, but with only a few objects to stay on top of things. 

In [12]:
test_arc = read_JSON_as_list('./JSON/test_architecture.json')
test_objects = get_all_objects(test_arc)
test_connections = get_all_connections(test_arc)

In [13]:
for obj in test_objects:
    print(obj[1][0][1])

Linear Spatial Template
Projection
Projection 2
Spatial Template
Group 1.Component Multiply
Group 1.Neural Field
Group 1.Static Gain
Group 2.Gauss Input
Group 2.Neural Field 2


In [14]:
test_connections

[[('source', 'Group 2.Neural Field 2.sigmoided activation'),
  ('target', 'Projection.input')],
 [('source', 'Group 2.Neural Field 2.sigmoided activation'),
  ('target', 'Projection 2.input')],
 [('source', 'Linear Spatial Template.spatial pattern'),
  ('target', 'Group 1.Component Multiply.operands')],
 [('source', 'Spatial Template.spatial pattern'),
  ('target', 'Group 1.Component Multiply.operands')],
 [('source', 'Spatial Template.spatial pattern'),
  ('target', 'Group 1.Static Gain.input')],
 [('source', 'Group 1.Neural Field.sigmoided activation'),
  ('target', 'Group 2.Neural Field 2.input')],
 [('source', 'Group 1.Component Multiply.product'),
  ('target', 'Group 1.Neural Field.input')],
 [('source', 'Group 1.Static Gain.output'),
  ('target', 'Group 1.Neural Field.input')],
 [('source', 'Group 2.Gauss Input.Gauss input'),
  ('target', 'Group 2.Neural Field 2.input')]]